In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation
from scipy.stats import pearsonr

In [3]:
ratings_filtered = pd.read_csv('ratings_data.csv')

# Data Preprocessing

In [5]:
ratings_filtered.groupby('user_id')[['book_id']].count().min()
ratings_filtered.groupby('user_id')[['book_id']].count().max()
ratings_filtered.groupby('user_id')[['book_id']].count().mean()

ratings_filtered.groupby('book_id')[['user_id']].count().min()
ratings_filtered.groupby('book_id')[['user_id']].count().max()
ratings_filtered.groupby('book_id')[['user_id']].count().mean()


ratings_filtered.shape
ratings_filtered.user_id.nunique()
ratings_filtered.book_id.nunique()

book_id    43
dtype: int64

book_id    110
dtype: int64

book_id    82.077457
dtype: float64

user_id    8
dtype: int64

user_id    219
dtype: int64

user_id    26.596676
dtype: float64

(153649, 4)

1872

5777

In [6]:
# r_temp = ratings_filtered.copy(deep=True)

In [7]:
# ratings_filtered.head()
# ratings_filtered.shape

,Unnamed: 0,user_id,book_id,rating
0,1510,103,867,5
1,1512,103,287,5
2,1513,103,2860,5
3,1514,103,397,4
4,1605,103,2027,5


(153649, 4)

In [8]:
# ratings_filtered['test'] = 0
# ratings_filtered.sort_values(by='user_id', inplace=True)

# # user similarity
# import time

# start = time.time()


# c = 0
# u = 0
# for index, row in ratings_filtered.iterrows():
#     if(c==0):
#         row['test'] = 1
#         c += 1
#         u = row['user_id']
#     else:
#         if(row['user_id']!=u):
#             c += 1
#             u = row['user_id']
#             row['test'] = 1
#     if(c==500):
#         break
    

# end = time.time()
# print(end - start)

# train = ratings_filtered[ratings_filtered['test']==0]
# test = ratings_filtered[ratings_filtered['test']==1]

# train.shape
# test.shape

5.928303480148315


(153149, 5)

(500, 5)

In [9]:
# train.groupby('user_id')[['book_id']].count().min()
# train.groupby('user_id')[['book_id']].count().max()
# train.groupby('user_id')[['book_id']].count().mean()

# train.groupby('book_id')[['user_id']].count().min()
# train.groupby('book_id')[['user_id']].count().max()
# train.groupby('book_id')[['user_id']].count().mean()

book_id    42
dtype: int64

book_id    109
dtype: int64

book_id    81.810363
dtype: float64

user_id    7
dtype: int64

user_id    219
dtype: int64

user_id    26.510126
dtype: float64

In [4]:
train = pd.read_csv('train_1106.csv')
test = pd.read_csv('test_1106.csv')

In [5]:
train.shape
test.shape

(153149, 6)

(500, 6)

# Collaborative Filtering

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation
from scipy.stats import pearsonr

In [7]:
user_item = pd.pivot_table(train, index='book_id', columns='user_id', values='rating')

In [8]:
user_item.shape
train.shape
test.shape

(5777, 1872)

(153149, 6)

(500, 6)

In [9]:
user_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5777 entries, 44 to 9995
Columns: 1872 entries, 67 to 53412
dtypes: float64(1872)
memory usage: 82.6 MB


In [10]:
user_item = user_item.add_prefix('user_')
user_item = user_item.set_index('book_'+user_item.index.astype(str))
user_item.head()

user_id,user_67,user_103,user_128,user_135,user_137,user_149,user_174,user_245,user_254,user_326,...,user_53280,user_53287,user_53298,user_53306,user_53319,user_53325,user_53341,user_53387,user_53398,user_53412
book_44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,5.0,NaN,5.0,NaN,NaN
book_68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
book_74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
book_77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
book_78,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN


In [11]:
data = user_item.copy(deep= True)

In [12]:
data.head()
data.shape

user_id,user_67,user_103,user_128,user_135,user_137,user_149,user_174,user_245,user_254,user_326,...,user_53280,user_53287,user_53298,user_53306,user_53319,user_53325,user_53341,user_53387,user_53398,user_53412
book_44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,5.0,NaN,5.0,NaN,NaN
book_68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
book_74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
book_77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
book_78,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN


(5777, 1872)

# Cosine Similarity

In [13]:
from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation

# user similarity
import time

start = time.time()

m = data.shape[0]
a = numpy.zeros(shape=(len(user_item.index),len(user_item.index)))
df = pd.DataFrame(a,columns=user_item.index.values)
df.set_index(user_item.index.values, inplace=True)

mask = numpy.isfinite(data)
# k = 0
for i in range(m-1):
    print('*******************************user number:'+str(i)+'*****************************************')
    for j in range(i + 1, m):
        curr = numpy.logical_and(mask.iloc[i], mask.iloc[j])
        if(curr.nunique()!=1): 
            u = data.iloc[i][curr]
            v = data.iloc[j][curr]
            df.iloc[i][j] = 1 - cosine(u,v)
            
end = time.time()
print(end - start)

*******************************user number:0*****************************************
*******************************user number:1*****************************************
*******************************user number:2*****************************************
*******************************user number:3*****************************************
*******************************user number:4*****************************************
*******************************user number:5*****************************************
*******************************user number:6*****************************************
*******************************user number:7*****************************************
*******************************user number:8*****************************************
*******************************user number:9*****************************************
*******************************user number:10*****************************************
*******************************user number:11********

*******************************user number:95*****************************************
*******************************user number:96*****************************************
*******************************user number:97*****************************************
*******************************user number:98*****************************************
*******************************user number:99*****************************************
*******************************user number:100*****************************************
*******************************user number:101*****************************************
*******************************user number:102*****************************************
*******************************user number:103*****************************************
*******************************user number:104*****************************************
*******************************user number:105*****************************************
*******************************user n

*******************************user number:189*****************************************
*******************************user number:190*****************************************
*******************************user number:191*****************************************
*******************************user number:192*****************************************
*******************************user number:193*****************************************
*******************************user number:194*****************************************
*******************************user number:195*****************************************
*******************************user number:196*****************************************
*******************************user number:197*****************************************
*******************************user number:198*****************************************
*******************************user number:199*****************************************
*******************************u

*******************************user number:283*****************************************
*******************************user number:284*****************************************
*******************************user number:285*****************************************
*******************************user number:286*****************************************
*******************************user number:287*****************************************
*******************************user number:288*****************************************
*******************************user number:289*****************************************
*******************************user number:290*****************************************
*******************************user number:291*****************************************
*******************************user number:292*****************************************
*******************************user number:293*****************************************
*******************************u

*******************************user number:377*****************************************
*******************************user number:378*****************************************
*******************************user number:379*****************************************
*******************************user number:380*****************************************
*******************************user number:381*****************************************
*******************************user number:382*****************************************
*******************************user number:383*****************************************
*******************************user number:384*****************************************
*******************************user number:385*****************************************
*******************************user number:386*****************************************
*******************************user number:387*****************************************
*******************************u

...

*******************************user number:5312*****************************************
*******************************user number:5313*****************************************
*******************************user number:5314*****************************************
*******************************user number:5315*****************************************
*******************************user number:5316*****************************************
*******************************user number:5317*****************************************
*******************************user number:5318*****************************************
*******************************user number:5319*****************************************
*******************************user number:5320*****************************************
*******************************user number:5321*****************************************
*******************************user number:5322*****************************************
*********************

*******************************user number:5405*****************************************
*******************************user number:5406*****************************************
*******************************user number:5407*****************************************
*******************************user number:5408*****************************************
*******************************user number:5409*****************************************
*******************************user number:5410*****************************************
*******************************user number:5411*****************************************
*******************************user number:5412*****************************************
*******************************user number:5413*****************************************
*******************************user number:5414*****************************************
*******************************user number:5415*****************************************
*********************

*******************************user number:5498*****************************************
*******************************user number:5499*****************************************
*******************************user number:5500*****************************************
*******************************user number:5501*****************************************
*******************************user number:5502*****************************************
*******************************user number:5503*****************************************
*******************************user number:5504*****************************************
*******************************user number:5505*****************************************
*******************************user number:5506*****************************************
*******************************user number:5507*****************************************
*******************************user number:5508*****************************************
*********************

*******************************user number:5591*****************************************
*******************************user number:5592*****************************************
*******************************user number:5593*****************************************
*******************************user number:5594*****************************************
*******************************user number:5595*****************************************
*******************************user number:5596*****************************************
*******************************user number:5597*****************************************
*******************************user number:5598*****************************************
*******************************user number:5599*****************************************
*******************************user number:5600*****************************************
*******************************user number:5601*****************************************
*********************

*******************************user number:5685*****************************************
*******************************user number:5686*****************************************
*******************************user number:5687*****************************************
*******************************user number:5688*****************************************
*******************************user number:5689*****************************************
*******************************user number:5690*****************************************
*******************************user number:5691*****************************************
*******************************user number:5692*****************************************
*******************************user number:5693*****************************************
*******************************user number:5694*****************************************
*******************************user number:5695*****************************************
*********************

In [14]:
# df.to_csv('cosine_similarity_item_1106.csv')
# df_corr.to_csv('correlaton_1102.csv')

In [15]:
df.head()
df.isnull().values.any()

,book_44,book_68,book_74,book_77,book_78,book_79,book_82,book_83,book_84,book_86,...,book_9907,book_9922,book_9923,book_9927,book_9938,book_9957,book_9961,book_9966,book_9978,book_9995
book_44,0.0,0.958986,0.954507,0.953608,0.931730,0.946756,0.913624,0.888275,0.947510,0.956593,...,1.000000,0.0,1.0,0.000000,0.000000,1.000000,1.0,0.888218,0.0,1.0
book_68,0.0,0.000000,0.963491,0.968500,0.936003,0.928586,0.950819,0.955125,0.926396,0.970363,...,0.992062,1.0,0.0,0.968277,1.000000,1.000000,0.0,0.000000,1.0,0.0
book_74,0.0,0.000000,0.000000,0.959860,0.973563,0.821025,0.945507,0.919365,0.925292,0.979862,...,0.000000,0.0,0.0,0.000000,0.989949,1.000000,0.0,0.000000,1.0,1.0
book_77,0.0,0.000000,0.000000,0.000000,0.946782,0.952840,0.897850,0.922109,0.976778,0.959360,...,1.000000,0.0,0.0,0.000000,0.000000,0.994692,0.0,0.000000,0.0,1.0
book_78,0.0,0.000000,0.000000,0.000000,0.000000,0.953198,0.932532,0.967782,0.968132,0.946240,...,1.000000,0.0,0.0,1.000000,0.000000,0.980581,1.0,1.000000,0.0,0.0


False

# Correlation

In [16]:
from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation

# user similarity
import time

start = time.time()

m = data.shape[0]
a = numpy.zeros(shape=(len(user_item.index),len(user_item.index)))

df_corr = pd.DataFrame(a,columns=user_item.index.values)
df_corr.set_index(user_item.index.values, inplace=True)
mask = numpy.isfinite(data)
for i in range(m-1):
    print('*******************************user number:'+str(i)+'*****************************************')
    for j in range(i + 1, m):
        curr = numpy.logical_and(mask.iloc[i], mask.iloc[j])
        if(curr.nunique()!=1): 
#             print('######################Found it! Phew!!'+str(i)+'and'+str(j)+'################################')
            u = data.iloc[i][curr]
            v = data.iloc[j][curr]
            df_corr.iloc[i][j] = 1-correlation(u, v)

end = time.time()
print(end - start)

*******************************user number:0*****************************************
*******************************user number:1*****************************************
*******************************user number:2*****************************************
*******************************user number:3*****************************************
*******************************user number:4*****************************************
*******************************user number:5*****************************************
*******************************user number:6*****************************************
*******************************user number:7*****************************************
*******************************user number:8*****************************************
*******************************user number:9*****************************************
*******************************user number:10*****************************************
*******************************user number:11********

*******************************user number:95*****************************************
*******************************user number:96*****************************************
*******************************user number:97*****************************************
*******************************user number:98*****************************************
*******************************user number:99*****************************************
*******************************user number:100*****************************************
*******************************user number:101*****************************************
*******************************user number:102*****************************************
*******************************user number:103*****************************************
*******************************user number:104*****************************************
*******************************user number:105*****************************************
*******************************user n

*******************************user number:189*****************************************
*******************************user number:190*****************************************
*******************************user number:191*****************************************
*******************************user number:192*****************************************
*******************************user number:193*****************************************
*******************************user number:194*****************************************
*******************************user number:195*****************************************
*******************************user number:196*****************************************
*******************************user number:197*****************************************
*******************************user number:198*****************************************
*******************************user number:199*****************************************
*******************************u

*******************************user number:283*****************************************
*******************************user number:284*****************************************
*******************************user number:285*****************************************
*******************************user number:286*****************************************
*******************************user number:287*****************************************
*******************************user number:288*****************************************
*******************************user number:289*****************************************
*******************************user number:290*****************************************
*******************************user number:291*****************************************
*******************************user number:292*****************************************
*******************************user number:293*****************************************
*******************************u

*******************************user number:377*****************************************
*******************************user number:378*****************************************
*******************************user number:379*****************************************
*******************************user number:380*****************************************
*******************************user number:381*****************************************
*******************************user number:382*****************************************
*******************************user number:383*****************************************
*******************************user number:384*****************************************
*******************************user number:385*****************************************
*******************************user number:386*****************************************
*******************************user number:387*****************************************
*******************************u

...

*******************************user number:5312*****************************************
*******************************user number:5313*****************************************
*******************************user number:5314*****************************************
*******************************user number:5315*****************************************
*******************************user number:5316*****************************************
*******************************user number:5317*****************************************
*******************************user number:5318*****************************************
*******************************user number:5319*****************************************
*******************************user number:5320*****************************************
*******************************user number:5321*****************************************
*******************************user number:5322*****************************************
*********************

*******************************user number:5406*****************************************
*******************************user number:5407*****************************************
*******************************user number:5408*****************************************
*******************************user number:5409*****************************************
*******************************user number:5410*****************************************
*******************************user number:5411*****************************************
*******************************user number:5412*****************************************
*******************************user number:5413*****************************************
*******************************user number:5414*****************************************
*******************************user number:5415*****************************************
*******************************user number:5416*****************************************
*********************

*******************************user number:5499*****************************************
*******************************user number:5500*****************************************
*******************************user number:5501*****************************************
*******************************user number:5502*****************************************
*******************************user number:5503*****************************************
*******************************user number:5504*****************************************
*******************************user number:5505*****************************************
*******************************user number:5506*****************************************
*******************************user number:5507*****************************************
*******************************user number:5508*****************************************
*******************************user number:5509*****************************************
*********************

*******************************user number:5593*****************************************
*******************************user number:5594*****************************************
*******************************user number:5595*****************************************
*******************************user number:5596*****************************************
*******************************user number:5597*****************************************
*******************************user number:5598*****************************************
*******************************user number:5599*****************************************
*******************************user number:5600*****************************************
*******************************user number:5601*****************************************
*******************************user number:5602*****************************************
*******************************user number:5603*****************************************
*********************

*******************************user number:5686*****************************************
*******************************user number:5687*****************************************
*******************************user number:5688*****************************************
*******************************user number:5689*****************************************
*******************************user number:5690*****************************************
*******************************user number:5691*****************************************
*******************************user number:5692*****************************************
*******************************user number:5693*****************************************
*******************************user number:5694*****************************************
*******************************user number:5695*****************************************
*******************************user number:5696*****************************************
*********************

In [17]:
df_corr.isnull().values.any()
df_corr.fillna(0,inplace=True)
df_corr.head()
df_corr.isnull().values.any()

True

,book_44,book_68,book_74,book_77,book_78,book_79,book_82,book_83,book_84,book_86,...,book_9907,book_9922,book_9923,book_9927,book_9938,book_9957,book_9961,book_9966,book_9978,book_9995
book_44,0.0,0.306478,0.333894,0.357232,0.211333,0.394118,-0.148608,-0.252485,0.098131,0.438853,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
book_68,0.0,0.000000,0.139598,0.246988,-0.161412,-0.083241,0.268251,0.236709,-0.129914,0.444302,...,0.866025,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
book_74,0.0,0.000000,0.000000,0.323618,-0.258199,-0.297650,-0.038837,-0.152675,-0.577350,0.627495,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
book_77,0.0,0.000000,0.000000,0.000000,0.169791,0.259745,-0.317495,0.140886,0.076799,0.324001,...,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
book_78,0.0,0.000000,0.000000,0.000000,0.000000,0.428098,0.218318,0.485451,0.476435,0.286255,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


False

In [18]:
# df_corr.to_csv('correlation_item_1106.csv')

In [26]:
# df_corr.to_csv('correlation_1106.csv')

# Testing

In [82]:
# df = pd.read_csv('cosine_similarity_1106.csv')
# df.rename(columns={'Unnamed: 0':'user_id'},inplace=True)
# df.set_index('user_id',inplace=True)

In [83]:
# df_corr = pd.read_csv('correlation_1106.csv')
# df_corr.rename(columns={'Unnamed: 0':'user_id'},inplace=True)
# df_corr.set_index('user_id',inplace=True)

In [84]:
# test = pd.read_csv('test_1106.csv')#('test_1102_idea1.csv')
# test.rename(columns={'Unnamed: 0.1':'id'},inplace=True)
# test.set_index('id',inplace=True)

In [85]:
# train = pd.read_csv('train_1106.csv')
# train.rename(columns={'Unnamed: 0.1':'id'},inplace=True)
# train.set_index('id',inplace=True)

In [35]:
test = pd.read_csv('test_cluster_0_1106.csv')#('test_1102_idea1.csv')
test.rename(columns={'Unnamed: 0':'id'},inplace=True)
test.set_index('id',inplace=True)

In [36]:
train.shape
test.shape
test.head()

(153149, 6)

(65, 3)

,user_id,book_id,rating
id,,,
0,137,2123,2
1,516,416,4
2,524,3309,3
3,681,174,4
4,1009,101,4


In [37]:
# remove the user not in train for cluster 0 
test_orig = test.copy(deep=True)
test = test[test['user_id']!=10324]

In [38]:
# book_ids which are rated by user1, book_ids not rated by user1 but rated by others
test['pred_rating_cos'] = 0.0
test['pred_rating_corr'] = 0.0
test['mean_rating'] = 0.0

for index, row in test.iterrows():
        print('*********************user:'+str(index)+'***************************')
        user = row['user_id']
        book = row['book_id']
        book_col = 'book_'+str(int(book))  #book to rate


        book_list = train[train['user_id']==user] #.drop(columns='user_id')    #books_read
        book_list["book_id"] = book_list["book_id"].map(lambda x : 'book_'+str(x))

        df_user = train[train['user_id']==user]

        test['mean_rating'][index] = df_user['rating'].mean()

        book_list["mean_rating"] = train[train['user_id']==user]["rating"].mean()
        book_list["adjusted_rating"] = book_list["rating"] - book_list["mean_rating"]

        book_list['similarity_cosine'] = book_list["book_id"].map(lambda x: df.loc[book_col,x] if df.loc[book_col,x]!=0 else df.loc[x,book_col])   
        book_list['similarity_correlation'] = book_list["book_id"].map(lambda x: df_corr.loc[book_col,x] if df_corr.loc[book_col,x]!=0 else df_corr.loc[x,book_col])
        
        # weighted similarity
        cos_sum = book_list['similarity_cosine'].abs().sum()
        corr_sum = book_list['similarity_correlation'].abs().sum()
        book_list['similarity_cosine'] = book_list['similarity_cosine'].map(lambda x: float(x)/cos_sum)
        book_list['similarity_correlation'] = book_list['similarity_correlation'].map(lambda x: float(x)/corr_sum)
        
        rating_list_cos = [row['similarity_cosine']*row['adjusted_rating'] for index, row in book_list.iterrows()]
        rating_list_corr = [row['similarity_correlation']*row['adjusted_rating'] for index, row in book_list.iterrows()]        

        test['pred_rating_cos'][index] = test['mean_rating'][index] + sum(rating_list_cos) #book_list['similarity_cosine'].dot(book_list['similarity_cosine'])

        test['pred_rating_corr'][index] = test['mean_rating'][index] + sum(rating_list_corr) #book_list['similarity_correlation'].dot(book_list['similarity_correlation'])


        test['error_cos'] = abs(test['pred_rating_cos'] - test['rating'])
        test['error_corr'] = abs(test['pred_rating_corr'] - test['rating'])

*********************user:0***************************
*********************user:1***************************
*********************user:2***************************
*********************user:3***************************
*********************user:4***************************
*********************user:5***************************
*********************user:6***************************
*********************user:7***************************
*********************user:8***************************
*********************user:9***************************
*********************user:10***************************
*********************user:11***************************
*********************user:12***************************
*********************user:13***************************
*********************user:14***************************
*********************user:15***************************
*********************user:16***************************
*********************user:17***************************
**

In [39]:
test['error_cos_sq'] = test['error_cos']*test['error_cos']
test['error_corr_sq'] = test['error_corr']*test['error_corr']

In [40]:
train.head()

,Unnamed: 0,Unnamed: 0.1,user_id,book_id,rating,test
0,25233,1037684,67,1523,3,0
1,24969,1024564,67,1481,3,0
2,24968,1024562,67,564,3,0
3,24967,1024560,67,435,4,0
4,23514,968609,67,106,5,0


In [41]:
print('MAE with mean rating is:'+str( float( (abs( test['mean_rating']-test['rating'] )).sum() ) /test.shape[0]))
print('MAE with Cosine Similarity is:'+str(float(test['error_cos'].sum())/test.shape[0]))
print('MAE with Correlation is:'+str(float(test['error_corr'].sum())/test.shape[0]))
print('\n')

print('MSE with mean rating is:'+str( float( (( test['mean_rating']-test['rating'] )*( test['mean_rating']-test['rating'] )).sum() ) /test.shape[0]))
print('MSE with Cosine Similarity is:'+str(float(test['error_cos_sq'].sum())/test.shape[0]))
print('MSE with Correlation is:'+str(float(test['error_corr_sq'].sum())/test.shape[0]))     

MAE with mean rating is:0.6712791682489818
MAE with Cosine Similarity is:0.6372754406139849
MAE with Correlation is:0.6161648861894298


MSE with mean rating is:0.688032817965408
MSE with Cosine Similarity is:0.6412619948360264
MSE with Correlation is:0.6690795000367287


In [42]:
# test.to_csv('train_cluster_0_results_1106_item_based.csv')

# Recommendation

In [62]:
df_cos = df.copy(deep=True)
df = df.copy(deep=True) # or df_corr

In [63]:
def find_n_neighbours(df,n):
    df = df.apply(lambda x: pd.Series(x.abs().sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [98]:
# top 10 neighbours for each user
sim_user_n_u = find_n_neighbours(df,20) # or df_corr
sim_user_n_u.head()
sim_user_n_u.shape

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20
book_44,book_9995,book_1076,book_4001,book_3997,book_1058,book_1059,book_3990,book_3976,book_3975,book_3973,book_3968,book_3953,book_3952,book_3950,book_3949,book_3947,book_3937,book_4007,book_4010,book_4014
book_68,book_2897,book_5158,book_3101,book_5165,book_5167,book_3100,book_5173,book_1424,book_3096,book_5179,book_806,book_3095,book_5184,book_9978,book_3081,book_3080,book_800,book_3076,book_1432,book_5201
book_74,book_9995,book_4139,book_1294,book_4156,book_1299,book_1300,book_4142,book_4140,book_4138,book_1290,book_1303,book_1306,book_1307,book_4128,book_4127,book_1310,book_4164,book_1289,book_4234,book_4216
book_77,book_9995,book_3512,book_3527,book_3526,book_3524,book_3522,book_3520,book_3518,book_3517,book_3509,book_3535,book_3508,book_3505,book_3494,book_3493,book_3491,book_3490,book_3485,book_3534,book_3537
book_78,book_3088,book_5499,book_1278,book_4074,book_1281,book_4067,book_4066,book_4065,book_1283,book_4063,book_1289,book_4052,book_4050,book_4046,book_4043,book_4042,book_1290,book_1277,book_4079,book_1276


(5777, 20)

In [99]:
# take user input
n = int(input("Enter the user id of interest : "))
user = 'user_'+str(n)
print(user)

Enter the user id of interest : 67
user_67


In [100]:
user_mean = train[train['user_id'] == n]['rating'].mean()

# get the list of potential books for recommendation
books_read = train[train['user_id']==n]['book_id']
df_book = train[train['user_id']==n]
df_book['adjusted_rating'] = df_book['rating'] - user_mean
df_book['book_idx'] = df_book.book_id.map(lambda x: 'book_'+str(x))

sim_books = []
for b in books_read:
    for p in sim_user_n_u.loc['book_'+str(b)]:
        sim_books.append(p)
sim_books = [n for n in sim_books if n not in books_read]

books_ratings_diff = {}
for book in sim_books:
    df_book['similarity'] = df_book.book_idx.map(lambda x: df.loc[book,x] if df.loc[book,x]!=0 else df.loc[x,book])
    df_book['adj_similarity'] = df_book['similarity'] / df_book['similarity'].abs().sum()
    df_book['rating_comp'] = df_book['adj_similarity']*df_book['adjusted_rating']
    books_ratings_diff[book] = df_book['rating_comp'].sum()

In [101]:
recommend_books = pd.DataFrame(columns=['book_id','rating_difference'])
recommend_books['book_id'] = [key for key in books_ratings_diff.keys()]
recommend_books['rating_difference'] = recommend_books['book_id'].map(lambda x: books_ratings_diff[x])
recommend_books.sort_values(by='rating_difference', ascending=False, inplace=True)

In [102]:
recommend_books['book_id'] = recommend_books.book_id.map(lambda x: int(x.split('_')[1]))
books = pd.read_csv('books.csv')
recommend_df = pd.merge(recommend_books, books, on='book_id', how='inner')[['book_id','rating_difference','title','authors']]

In [103]:
recommend_df

,book_id,rating_difference,title,authors
0,3820,0.716593,The Call of Cthulhu and Other Weird Stories,"H.P. Lovecraft, S.T. Joshi"
1,4201,0.700986,Officer Buckle & Gloria,Peggy Rathmann
2,2670,0.670853,Where Eagles Dare,"Alistair MacLean, Clint Willis"
3,6807,0.669655,Strange Highways,Dean Koontz
4,3404,0.663667,The Histories,"Herodotus, Aubrey de Sélincourt, John M. Marin..."
5,4220,0.645042,Underworld,Don DeLillo
6,5496,0.639886,Writing Down the Bones: Freeing the Writer Within,Natalie Goldberg
7,4795,0.623064,Night Chills,Dean Koontz
8,4228,0.623064,"Preacher, Volume 2: Until the End of the World","Garth Ennis, Steve Dillon"
9,3246,0.621820,"The Rise of Endymion (Hyperion Cantos, #4)",Dan Simmons


In [104]:
test[test['user_id']==n].merge(books, on='book_id')[['title','rating','authors']].sort_values(by='rating', ascending=False)
recommend_df[recommend_df['title']=='The Virgin Suicides']

,title,rating,authors


,book_id,rating_difference,title,authors


In [105]:
train[train['user_id']==n].merge(books, on='book_id')[['title','rating','authors']].sort_values(by='rating', ascending=False)

,title,rating,authors
30,The Prisoner of Heaven (The Cemetery of Forgot...,5,Carlos Ruiz Zafón
45,"Unbroken: A World War II Story of Survival, Re...",5,Laura Hillenbrand
28,11/22/1963,5,Stephen King
33,Where Things Come Back,5,John Corey Whaley
34,Between Shades of Gray,5,Ruta Sepetys
69,The Guernsey Literary and Potato Peel Pie Society,5,"Mary Ann Shaffer, Annie Barrows"
39,The Night Circus,5,Erin Morgenstern
31,Let the Great World Spin,5,Colum McCann
4,Bossypants,5,Tina Fey
53,A Fine Balance,5,Rohinton Mistry
